### Lab2 : Working with Spark SQL

We will review :

1. Loading CSV file formats using SparkSession
2. Creating DataFrame without inferring Schema 
3. Creating DataFrame inferring Schema 
4. Creating DataFrame using databricks library that help inferring the schema
5. Creating partitioned parquet files format
6. Doing some preliminary analysis of this dataset

Dataset :

* Air flight data - subset of ~ few GBs ( Include reference )

Dataset path (in the cluster) :

* /data/shared/spark/flight_data/csv_small


In [23]:
# First Let's start by :
# 1. Definining SPARK_HOME variable 
# 2. Using findspark to  let us work with Spark installation in the cluster

In [24]:
import os
os.environ['SPARK_HOME']="/usr/hdp/current/spark2-client"

In [25]:
import findspark
findspark.init()
import pyspark

In [26]:
# Create a SparkSession
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Spark-SQL-Lab2") \
    .getOrCreate()

In [27]:
dataset_path="/data/shared/spark/flight_data/csv_small/"

In [28]:
# Read in one of the data files into a data frame
df = spark.read \
    .csv("file://"+dataset_path+"On_Time_On_Time_Performance_2014_9.csv")

### Now check the data schema

In [29]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)
 |-- _c12: string (nullable = true)
 |-- _c13: string (nullable = true)
 |-- _c14: string (nullable = true)
 |-- _c15: string (nullable = true)
 |-- _c16: string (nullable = true)
 |-- _c17: string (nullable = true)
 |-- _c18: string (nullable = true)
 |-- _c19: string (nullable = true)
 |-- _c20: string (nullable = true)
 |-- _c21: string (nullable = true)
 |-- _c22: string (nullable = true)
 |-- _c23: string (nullable = true)
 |-- _c24: string (nullable = true)
 |-- _c25: string (nullable = true)
 |-- _c26: string (nullable = true)
 |-- _c27: string (nullable = tru

* Ok , but the column names are not very telling. 
* How to improve this? , by telling Spark to use the header ( if exists )

In [30]:
df = spark.read \
    .option("header", "true") \
    .csv("file://"+dataset_path+"On_Time_On_Time_Performance_2014_9.csv")

In [31]:
df.printSchema()

root
 |-- Year: string (nullable = true)
 |-- Quarter: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- DayofMonth: string (nullable = true)
 |-- DayOfWeek: string (nullable = true)
 |-- FlightDate: string (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- AirlineID: string (nullable = true)
 |-- Carrier: string (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- FlightNum: string (nullable = true)
 |-- OriginAirportID: string (nullable = true)
 |-- OriginAirportSeqID: string (nullable = true)
 |-- OriginCityMarketID: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- OriginCityName: string (nullable = true)
 |-- OriginState: string (nullable = true)
 |-- OriginStateFips: string (nullable = true)
 |-- OriginStateName: string (nullable = true)
 |-- OriginWac: string (nullable = true)
 |-- DestAirportID: string (nullable = true)
 |-- DestAirportSeqID: string (nullable = true)
 |-- DestCityMarketID: string (nullable = true)
 |-

* Still one caveat though , all values are interpreted as StringType() While some of them (actually most), are of numeric nature ( e.g ) Year , Month , Flight Number
* How to improve this ?, by telling Spark to infer the Schema of the data

In [32]:
df = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("file://"+dataset_path+"On_Time_On_Time_Performance_2014_9.csv")

In [33]:
df.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Quarter: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- FlightDate: timestamp (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- AirlineID: integer (nullable = true)
 |-- Carrier: string (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- FlightNum: integer (nullable = true)
 |-- OriginAirportID: integer (nullable = true)
 |-- OriginAirportSeqID: integer (nullable = true)
 |-- OriginCityMarketID: integer (nullable = true)
 |-- Origin: string (nullable = true)
 |-- OriginCityName: string (nullable = true)
 |-- OriginState: string (nullable = true)
 |-- OriginStateFips: integer (nullable = true)
 |-- OriginStateName: string (nullable = true)
 |-- OriginWac: integer (nullable = true)
 |-- DestAirportID: integer (nullable = true)
 |-- DestAirportSeqID: integer (nullable = true)
 |-- DestCityMarketID: integer (nu

* Now that a way to have data with schema , let's read in some more data 

In [34]:
# READ IN ONE YEAR WORTH OF DATA : ~ 2.5GB
df_small = spark.read \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .csv("file://"+dataset_path+"*csv") 
df_small.cache()

DataFrame[Year: int, Quarter: int, Month: int, DayofMonth: int, DayOfWeek: int, FlightDate: timestamp, UniqueCarrier: string, AirlineID: int, Carrier: string, TailNum: string, FlightNum: int, OriginAirportID: int, OriginAirportSeqID: int, OriginCityMarketID: int, Origin: string, OriginCityName: string, OriginState: string, OriginStateFips: int, OriginStateName: string, OriginWac: int, DestAirportID: int, DestAirportSeqID: int, DestCityMarketID: int, Dest: string, DestCityName: string, DestState: string, DestStateFips: int, DestStateName: string, DestWac: int, CRSDepTime: int, DepTime: int, DepDelay: double, DepDelayMinutes: double, DepDel15: double, DepartureDelayGroups: int, DepTimeBlk: string, TaxiOut: double, WheelsOff: int, WheelsOn: int, TaxiIn: double, CRSArrTime: int, ArrTime: int, ArrDelay: double, ArrDelayMinutes: double, ArrDel15: double, ArrivalDelayGroups: int, ArrTimeBlk: string, Cancelled: double, CancellationCode: string, Diverted: double, CRSElapsedTime: double, ActualE

* Save this dataframe in parquet (columnar) format for boost in loading performance
* In order to do we want to 'be clever' and partition the data by specific atributes , in this case
* Year and Month

In [38]:
# Save the data into my HOME
my_home=os.environ['HOME']
out_dir="airline_data"
df_small.write.partitionBy(
        "Year","Month"
    ).parquet(
        my_home
        +'/'
        + out_dir
        ,
        mode='overwrite'
    )
print('Done!')

Py4JJavaError: An error occurred while calling o237.parquet.
: org.apache.hadoop.security.AccessControlException: Permission denied: user=aabreu, access=WRITE, inode="/data/home/aabreu/airline_data/_temporary/0":hdfs:hdfs:drwxr-xr-x
	at org.apache.hadoop.hdfs.server.namenode.FSPermissionChecker.check(FSPermissionChecker.java:319)
	at org.apache.hadoop.hdfs.server.namenode.FSPermissionChecker.check(FSPermissionChecker.java:292)
	at org.apache.hadoop.hdfs.server.namenode.FSPermissionChecker.checkPermission(FSPermissionChecker.java:213)
	at org.apache.hadoop.hdfs.server.namenode.FSPermissionChecker.checkPermission(FSPermissionChecker.java:190)
	at org.apache.hadoop.hdfs.server.namenode.FSDirectory.checkPermission(FSDirectory.java:1955)
	at org.apache.hadoop.hdfs.server.namenode.FSDirectory.checkPermission(FSDirectory.java:1939)
	at org.apache.hadoop.hdfs.server.namenode.FSDirectory.checkAncestorAccess(FSDirectory.java:1922)
	at org.apache.hadoop.hdfs.server.namenode.FSDirMkdirOp.mkdirs(FS

In [42]:
# Read CSV data into a dictionary of DataFrame : try to infer schema directly from the data
import itertools
year_list = ['2014']
month_list = ['1','2','3','4','5','6','7','8','9','10','11','12']

dict_df = {}

for (year_str,month_str) in list(itertools.product(year_list,month_list)):
    year_month_str = '%s_%s'%(year_str,month_str)
    print('Reading input data for year:%s month:%s'%(year_str,month_str))
    df = spark.read \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .csv("file://"+dataset_path+"On_Time_On_Time_Performance_%s.csv"%(year_month_str))  
    df.cache()
    dict_df[year_month_str]=df
print('Done!')

Reading input data for year:2014 month:1
Reading input data for year:2014 month:2
Reading input data for year:2014 month:3
Reading input data for year:2014 month:4
Reading input data for year:2014 month:5
Reading input data for year:2014 month:6
Reading input data for year:2014 month:7
Reading input data for year:2014 month:8
Reading input data for year:2014 month:9
Reading input data for year:2014 month:10
Reading input data for year:2014 month:11
Reading input data for year:2014 month:12
Reading input data for year:2015 month:1
Reading input data for year:2015 month:2
Reading input data for year:2015 month:3
Reading input data for year:2015 month:4
Reading input data for year:2015 month:5
Reading input data for year:2015 month:6
Reading input data for year:2015 month:7
Reading input data for year:2015 month:8
Reading input data for year:2015 month:9
Reading input data for year:2015 month:10
Reading input data for year:2015 month:11
Reading input data for year:2015 month:12
Done!


In [14]:
# Create DataFrame : inferring Schema Using Databricks library
df_with_dbricks = spark.read \
    .format("com.databricks.spark.csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("file://"+dataset_path+"On_Time_On_Time_Performance_2014_9.csv")

In [15]:
df_with_dbricks.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Quarter: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- FlightDate: timestamp (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- AirlineID: integer (nullable = true)
 |-- Carrier: string (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- FlightNum: integer (nullable = true)
 |-- OriginAirportID: integer (nullable = true)
 |-- OriginAirportSeqID: integer (nullable = true)
 |-- OriginCityMarketID: integer (nullable = true)
 |-- Origin: string (nullable = true)
 |-- OriginCityName: string (nullable = true)
 |-- OriginState: string (nullable = true)
 |-- OriginStateFips: integer (nullable = true)
 |-- OriginStateName: string (nullable = true)
 |-- OriginWac: integer (nullable = true)
 |-- DestAirportID: integer (nullable = true)
 |-- DestAirportSeqID: integer (nullable = true)
 |-- DestCityMarketID: integer (nu